In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
color_pal = sns.color_palette()
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [21]:
import sqlite3

conn = sqlite3.connect('../data/WARP.db')
df = pd.read_sql_query("SELECT * FROM master_warp", conn)
conn.close()
print(df.columns)

Index(['Price', 'target_datetime', 'Load', 'shortwave_radiation',
       'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation',
       'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin',
       'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover',
       'weekday_sin', 'hour_sin', 'weekday_cos'],
      dtype='object')


In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = [
    'Flow_NO','is_dst', 'hour_cos', 'hour_sin','month',
    'Load','is_non_working_day','shortwave_radiation', 'temperature_2m' 
] # excluding  'direct_normal_irradiance', 'diffuse_radiation','yearday_cos', 'yearday_sin', 'cloud_cover', 'Flow_GB',  'is_weekend',
target = 'Price'


# Safe datetime handling
if 'target_datetime' not in df.columns:
    print("'target_datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()
else:
    df['target_datetime'] = pd.to_datetime(df['target_datetime'])
    df = df.set_index('target_datetime')


# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-14 12:00", tz='UTC')
lag = timedelta(hours=36)
forecast_horizon = timedelta(hours=144)

# Store RMSEs
rmses = []

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag # check current time  
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, RMSE: {rmse:.3f}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.5f}")

# Calculate average RMSE over the first 30 runs
if len(rmses) >= 30:
    avg_rmse_30 = np.mean(rmses[:30])
    print(f"Average RMSE over first 30 runs: {avg_rmse_30:.5f}")
else:
    print("Less than 30 runs available to calculate average RMSE.")

Forecast origin: 2025-03-13 12:00:00+00:00, Predicting 2025-03-15 00:00:00+00:00 to 2025-03-21 00:00:00+00:00, RMSE: 0.020
Forecast origin: 2025-03-14 12:00:00+00:00, Predicting 2025-03-16 00:00:00+00:00 to 2025-03-22 00:00:00+00:00, RMSE: 0.024
Forecast origin: 2025-03-15 12:00:00+00:00, Predicting 2025-03-17 00:00:00+00:00 to 2025-03-23 00:00:00+00:00, RMSE: 0.032
Forecast origin: 2025-03-16 12:00:00+00:00, Predicting 2025-03-18 00:00:00+00:00 to 2025-03-24 00:00:00+00:00, RMSE: 0.030
Forecast origin: 2025-03-17 12:00:00+00:00, Predicting 2025-03-19 00:00:00+00:00 to 2025-03-25 00:00:00+00:00, RMSE: 0.033
Forecast origin: 2025-03-18 12:00:00+00:00, Predicting 2025-03-20 00:00:00+00:00 to 2025-03-26 00:00:00+00:00, RMSE: 0.032
Forecast origin: 2025-03-19 12:00:00+00:00, Predicting 2025-03-21 00:00:00+00:00 to 2025-03-27 00:00:00+00:00, RMSE: 0.029
Forecast origin: 2025-03-20 12:00:00+00:00, Predicting 2025-03-22 00:00:00+00:00 to 2025-03-28 00:00:00+00:00, RMSE: 0.025
Forecast origin:

In [ ]:
# no lag, 7 days forecast horizon. higly similar average RMSE to the 36 hours lag, 6 days forecast horizon.
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from datetime import timedelta

# Feature and target setup
features = [
    'Flow_NO','is_dst', 'hour_cos', 'hour_sin','month',
    'Load','is_non_working_day','shortwave_radiation', 'temperature_2m'
] # excluding  'direct_normal_irradiance', 'diffuse_radiation', 'cloud_cover', 'Flow_GB',  'is_weekend','yearday_cos', 'yearday_sin'
target = 'Price'


# Safe datetime handling
if 'target_datetime' not in df.columns:
    print("'target_datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()
else:
    df['target_datetime'] = pd.to_datetime(df['target_datetime'])
    df = df.set_index('target_datetime')


# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-14 12:00", tz='UTC')
lag = timedelta(hours=0)  # No lag
forecast_horizon = timedelta(hours=168)  # 7 days

# Store RMSEs
rmses = []

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag # check current time  
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Train and predict
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmses.append(rmse)

    print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, RMSE: {rmse:.3f}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.5f}")

# Calculate average RMSE over the first 30 runs
if len(rmses) >= 30:
    avg_rmse_30 = np.mean(rmses[:30])
    print(f"Average RMSE over first 30 runs: {avg_rmse_30:.5f}")
else:
    print("Less than 30 runs available to calculate average RMSE.")

'target_datetime' column not found in columns. Sorting by index instead.
Forecast origin: 2025-03-13 12:00:00+00:00, Predicting 2025-03-13 12:00:00+00:00 to 2025-03-20 12:00:00+00:00, RMSE: 0.019
Forecast origin: 2025-03-14 12:00:00+00:00, Predicting 2025-03-14 12:00:00+00:00 to 2025-03-21 12:00:00+00:00, RMSE: 0.020
Forecast origin: 2025-03-15 12:00:00+00:00, Predicting 2025-03-15 12:00:00+00:00 to 2025-03-22 12:00:00+00:00, RMSE: 0.027
Forecast origin: 2025-03-16 12:00:00+00:00, Predicting 2025-03-16 12:00:00+00:00 to 2025-03-23 12:00:00+00:00, RMSE: 0.030
Forecast origin: 2025-03-17 12:00:00+00:00, Predicting 2025-03-17 12:00:00+00:00 to 2025-03-24 12:00:00+00:00, RMSE: 0.030
Forecast origin: 2025-03-18 12:00:00+00:00, Predicting 2025-03-18 12:00:00+00:00 to 2025-03-25 12:00:00+00:00, RMSE: 0.030
Forecast origin: 2025-03-19 12:00:00+00:00, Predicting 2025-03-19 12:00:00+00:00 to 2025-03-26 12:00:00+00:00, RMSE: 0.031
Forecast origin: 2025-03-20 12:00:00+00:00, Predicting 2025-03-20 

In [25]:
# now with hyper parameter tuning using gridsearchCV
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from datetime import timedelta

# Feature and target setup
features = [
    'Flow_NO','is_dst', 'hour_cos', 'hour_sin','month',
    'Load','is_non_working_day','shortwave_radiation', 'temperature_2m' 
]
target = 'Price'

# Safe datetime handling
if 'target_datetime' not in df.columns:
    print("'target_datetime' column not found in columns. Sorting by index instead.")
    df = df.sort_index()
else:
    df['target_datetime'] = pd.to_datetime(df['target_datetime'])
    df = df.set_index('target_datetime')

# Forecast settings
start_date = pd.Timestamp("2025-03-13 12:00", tz='UTC')
end_date = pd.Timestamp("2025-05-14 12:00", tz='UTC')
lag = timedelta(hours=36)
forecast_horizon = timedelta(hours=144)

# Store RMSEs
rmses = []

# Extensive grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1],
    'colsample_bytree': [0.6, 0.8, 1],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2],
}

current_time = start_date
while current_time <= end_date:
    train_data = df[df.index < current_time]
    test_start = current_time + lag
    test_end = test_start + forecast_horizon
    test_data = df[(df.index >= test_start) & (df.index < test_end)]

    if test_data.empty:
        print(f"No test data for forecast starting at {current_time}")
        current_time += timedelta(days=1)
        continue

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    try:
        base_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            scoring='neg_root_mean_squared_error',
            cv=3,
            n_jobs=-1,
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmses.append(rmse)

        print(f"Forecast origin: {current_time}, Predicting {test_start} to {test_end}, "
              f"RMSE: {rmse:.3f}, Best params: {best_params}")

    except Exception as e:
        print(f"Error on forecast starting at {current_time}: {e}")

    current_time += timedelta(days=1)

# Summary
avg_rmse = np.mean(rmses)
print(f"\nAverage RMSE over {len(rmses)} runs: {avg_rmse:.5f}")

# Average over first 30 runs
if len(rmses) >= 30:
    avg_rmse_30 = np.mean(rmses[:30])
    print(f"Average RMSE over first 30 runs: {avg_rmse_30:.5f}")
else:
    print("Less than 30 runs available to calculate average RMSE.")

'target_datetime' column not found in columns. Sorting by index instead.
Forecast origin: 2025-03-13 12:00:00+00:00, Predicting 2025-03-15 00:00:00+00:00 to 2025-03-21 00:00:00+00:00, RMSE: 0.020, Best params: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1.5, 'subsample': 1}
Forecast origin: 2025-03-14 12:00:00+00:00, Predicting 2025-03-16 00:00:00+00:00 to 2025-03-22 00:00:00+00:00, RMSE: 0.024, Best params: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1.5, 'subsample': 0.6}
Forecast origin: 2025-03-15 12:00:00+00:00, Predicting 2025-03-17 00:00:00+00:00 to 2025-03-23 00:00:00+00:00, RMSE: 0.032, Best params: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'reg_alpha': 0.1, 'reg_lambda': 1, 'subsample': 1}
Forecast origin: 2025-03-16 12:00:00+00:00, Predicting 2025-03-18 00:

In [27]:
# Collect best_params from each run if available in rmse_results
best_params_list = []
for result in rmse_results:
    if 'best_params' in result:
        best_params_list.append(result['best_params'])

if best_params_list:
    best_params_df = pd.DataFrame(best_params_list)
    print("Best hyper-parameter values across runs (top 10 shown):")
    print(best_params_df.head(10))
    print("\nMost frequent values for each hyper-parameter:")
    print(best_params_df.mode().iloc[0])
else:
    print("No best_params found in rmse_results. Make sure to store 'best_params' in each run.")

No best_params found in rmse_results. Make sure to store 'best_params' in each run.


In [17]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# Dynamic path setup
import sys
from pathlib import Path

# Find project root dynamically
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir

# Add utils to path
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

# Define feature columns and target
COMMON_FEATURES = ['is_dst', 'hour_cos', 'hour_sin','month','is_non_working_day','shortwave_radiation', 'temperature_2m'
    ]
# list of all available features 'cloud_cover', 'direct_normal_irradiance', 'diffuse_radiation','hour_cos', 'hour_sin', 'is_dst', 'is_non_working_day','is_weekend', 'month', 'shortwave_radiation','temperature_2m', 'yearday_cos', 'yearday_sin'
TRAIN_ONLY_FEATURES = ['Load','Flow_NO']  # <- Only used if known ex post

TRAIN_FEATURES = COMMON_FEATURES + TRAIN_ONLY_FEATURES
TEST_FEATURES = COMMON_FEATURES


target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing XGBoost Model - RMSE per forecast day")
print("=" * 60)

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime').set_index('target_datetime')

        # Zorg dat run_date ook in UTC is
        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df.index <= run_date_utc]
        test_data = df[df.index > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=features + [target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Train model
        X_train = train_data[TRAIN_FEATURES]
        y_train = train_data[target]

        # Aanvullen met NaN-kolommen waar nodig
        for col in TRAIN_FEATURES:
            if col not in test_data.columns:
                test_data[col] = np.nan

        X_test = test_data[TRAIN_FEATURES]
        y_test = test_data[target]

        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(test_data),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(test_data)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {len(rmse_df)}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.2f}")
else:
    print("❌ No runs completed successfully")

2025-05-27 11:16:48,628 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:16:48,629 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-27 11:16:48,629 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-27 11:16:48,680 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen


🔍 Testing XGBoost Model - RMSE per forecast day


2025-05-27 11:16:48,841 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:16:48,856 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:16:48,856 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

Day 1: ✅ 167 test rows, Run: 03-15


2025-05-27 11:16:49,226 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:16:49,235 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:16:49,235 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

...

Day 27: ✅ 167 test rows, Run: 04-10


/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:16:57,404 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:16:57,404 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos', 'apparent_temperature', 'day_of_week',

Day 28: ✅ 167 test rows, Run: 04-11


2025-05-27 11:16:57,837 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:16:57,837 - build_training_set - INFO - 🧠 Actuals van 2025-01-30 00:00:00+00:00 t/m 2025-04-12 23:00:00+00:00
2025-05-27 11:16:57,838 - build_training_set - INFO - 📅 Forecast van run_date 2025-04-13 00:00:00+00:00, target range: 2025-04-13 00:00:00+00:00 → 2025-04-19 23:00:00+00:00
2025-05-27 11:16:57,855 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-27 11:16:57,995 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combin

Day 29: ✅ 167 test rows, Run: 04-12
Day 30: ✅ 167 test rows, Run: 04-13

📊 OVERALL RMSE - XGBoost Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                167  0.08
         2 2025-03-16                167  0.09
         3 2025-03-17                167  0.09
         4 2025-03-18                167  0.08
         5 2025-03-19                167  0.08
         6 2025-03-20                167  0.08
         7 2025-03-21                167  0.07
         8 2025-03-22                167  0.08
         9 2025-03-23                167  0.06
        10 2025-03-24                167  0.07
        11 2025-03-25                167  0.09
        12 2025-03-26                167  0.09
        13 2025-03-27                167  0.10
        14 2025-03-28                167  0.11
        15 2025-03-29                167  0.09
        16 2025-03-30                167  0.10
        17 2025-03-31                167  0.10
        18 2025-04-01      

In [23]:
# optimizing the code for feature selection 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# Dynamic path setup
import sys
from pathlib import Path
from sklearn.model_selection import GridSearchCV

# Find project root dynamically
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir

# Add utils to path
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

# Define feature columns and target
COMMON_FEATURES = ['is_dst', 'hour_cos', 'hour_sin','month','is_non_working_day','shortwave_radiation', 'temperature_2m'
    ]
# list of all available features 'cloud_cover', 'direct_normal_irradiance', 'diffuse_radiation','hour_cos', 'hour_sin', 'is_dst', 'is_non_working_day','is_weekend', 'month', 'shortwave_radiation','temperature_2m', 'yearday_cos', 'yearday_sin'
TRAIN_ONLY_FEATURES = ['Load','Flow_NO']  # <- Only used if known ex post

TRAIN_FEATURES = COMMON_FEATURES + TRAIN_ONLY_FEATURES
TEST_FEATURES = COMMON_FEATURES


target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing XGBoost Model - RMSE per forecast day")
print("=" * 60)

# GridSearch parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

# Loop over forecast days
for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime').set_index('target_datetime')

        # Zorg dat run_date ook in UTC is
        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df.index <= run_date_utc]
        test_data = df[df.index > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=TRAIN_FEATURES + [target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Train model
        X_train = train_data[TRAIN_FEATURES]
        y_train = train_data[target]

        # Aanvullen met NaN-kolommen waar nodig
        for col in TRAIN_FEATURES:
            if col not in test_data.columns:
                test_data[col] = np.nan

        X_test = test_data[TRAIN_FEATURES]
        y_test = test_data[target]

        base_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            scoring='neg_root_mean_squared_error',
            cv=3,
            n_jobs=-1,
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        print(f"Day {i+1}: 🧪 Best params: {grid_search.best_params_}")
        model = grid_search.best_estimator_

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(test_data),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(test_data)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {len(rmse_df)}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.2f}")
else:
    print("❌ No runs completed successfully")

2025-05-27 11:44:54,535 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:44:54,536 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-27 11:44:54,536 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-27 11:44:54,588 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen


🔍 Testing XGBoost Model - RMSE per forecast day


2025-05-27 11:44:54,822 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:44:54,834 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:44:54,834 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

Day 1: 🧪 Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1}
Day 1: ✅ 167 test rows, Run: 03-15


2025-05-27 11:44:58,841 - build_training_set - INFO - ✅ Opgeslagen als training_set in WARP.db
2025-05-27 11:44:58,842 - build_training_set - INFO - 🔒 Verbinding gesloten
2025-05-27 11:44:59,641 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:44:59,643 - build_training_set - INFO - 🧠 Actuals van 2025-01-03 00:00:00+00:00 t/m 2025-03-16 23:00:00+00:00
2025-05-27 11:44:59,644 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-17 00:00:00+00:00, target range: 2025-03-17 00:00:00+00:00 → 2025-03-23 23:00:00+00:00
2025-05-27 11:44:59,666 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-27 11:44:59,821 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no l

...

Day 27: 🧪 Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Day 27: ✅ 167 test rows, Run: 04-10


2025-05-27 11:45:22,498 - build_training_set - INFO - ✅ Opgeslagen als training_set in WARP.db
2025-05-27 11:45:22,499 - build_training_set - INFO - 🔒 Verbinding gesloten
2025-05-27 11:45:23,240 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:45:23,241 - build_training_set - INFO - 🧠 Actuals van 2025-01-29 00:00:00+00:00 t/m 2025-04-11 23:00:00+00:00
2025-05-27 11:45:23,241 - build_training_set - INFO - 📅 Forecast van run_date 2025-04-12 00:00:00+00:00, target range: 2025-04-12 00:00:00+00:00 → 2025-04-18 23:00:00+00:00
2025-05-27 11:45:23,257 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-27 11:45:23,404 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no l

Day 28: 🧪 Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Day 28: ✅ 167 test rows, Run: 04-11


2025-05-27 11:45:24,108 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:45:24,108 - build_training_set - INFO - 🧠 Actuals van 2025-01-30 00:00:00+00:00 t/m 2025-04-12 23:00:00+00:00
2025-05-27 11:45:24,108 - build_training_set - INFO - 📅 Forecast van run_date 2025-04-13 00:00:00+00:00, target range: 2025-04-13 00:00:00+00:00 → 2025-04-19 23:00:00+00:00
2025-05-27 11:45:24,125 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-27 11:45:24,280 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combin

Day 29: 🧪 Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Day 29: ✅ 167 test rows, Run: 04-12
Day 30: 🧪 Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Day 30: ✅ 167 test rows, Run: 04-13

📊 OVERALL RMSE - XGBoost Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                167  0.09
         2 2025-03-16                167  0.10
         3 2025-03-17                167  0.10
         4 2025-03-18                167  0.09
         5 2025-03-19                167  0.09
         6 2025-03-20                167  0.08
         7 2025-03-21                167  0.10
         8 2025-03-22                167  0.09
         9 2025-03-23                167  0.07
        10 2025-03-24                167  0.10
        11 2025-03-25                167  0.10
        12 2025-03-26                167  0.09
        

In [24]:
# andere setting hyperparameters in GridSearchCV


import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# Dynamic path setup
import sys
from pathlib import Path
from sklearn.model_selection import GridSearchCV

# Find project root dynamically
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir

# Add utils to path
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

# Define feature columns and target
COMMON_FEATURES = ['is_dst', 'hour_cos', 'hour_sin','month','is_non_working_day','shortwave_radiation', 'temperature_2m'
    ]
# list of all available features 'cloud_cover', 'direct_normal_irradiance', 'diffuse_radiation','hour_cos', 'hour_sin', 'is_dst', 'is_non_working_day','is_weekend', 'month', 'shortwave_radiation','temperature_2m', 'yearday_cos', 'yearday_sin'
TRAIN_ONLY_FEATURES = ['Load','Flow_NO']  # <- Only used if known ex post

TRAIN_FEATURES = COMMON_FEATURES + TRAIN_ONLY_FEATURES
TEST_FEATURES = COMMON_FEATURES


target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing XGBoost Model - RMSE per forecast day")
print("=" * 60)

# GridSearch parameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1],
    'colsample_bytree': [0.6, 0.8, 1],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2],
}

# Loop over forecast days
for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime').set_index('target_datetime')

        # Zorg dat run_date ook in UTC is
        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df.index <= run_date_utc]
        test_data = df[df.index > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=TRAIN_FEATURES + [target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Train model
        X_train = train_data[TRAIN_FEATURES]
        y_train = train_data[target]

        # Aanvullen met NaN-kolommen waar nodig
        for col in TRAIN_FEATURES:
            if col not in test_data.columns:
                test_data[col] = np.nan

        X_test = test_data[TRAIN_FEATURES]
        y_test = test_data[target]

        base_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            scoring='neg_root_mean_squared_error',
            cv=3,
            n_jobs=-1,
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        print(f"Day {i+1}: 🧪 Best params: {grid_search.best_params_}")
        model = grid_search.best_estimator_

        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(test_data),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(test_data)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - XGBoost Model")
    print("=" * 80)
    print(f"Successful runs: {len(rmse_df)}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.2f}")
else:
    print("❌ No runs completed successfully")

2025-05-27 11:49:28,630 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-27 11:49:28,632 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-27 11:49:28,635 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-27 11:49:28,729 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen


🔍 Testing XGBoost Model - RMSE per forecast day


2025-05-27 11:49:28,893 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:49:28,904 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:49:28,904 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

Day 1: 🧪 Best params: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.6}
Day 1: ✅ 167 test rows, Run: 03-15


2025-05-27 11:51:37,835 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 11:51:37,845 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 11:51:37,846 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

...

Day 28: 🧪 Best params: {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'reg_alpha': 0, 'reg_lambda': 1.5, 'subsample': 0.6}
Day 28: ✅ 167 test rows, Run: 04-11


2025-05-27 13:06:05,382 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 13:06:05,393 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 13:06:05,393 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

Day 29: 🧪 Best params: {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'reg_alpha': 0.1, 'reg_lambda': 1, 'subsample': 0.8}
Day 29: ✅ 167 test rows, Run: 04-12


2025-05-27 13:08:52,078 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
/Users/Twan/Library/Mobile Documents/com~apple~CloudDocs/Data Science/Data Projects EASI/ENEXIS/src/utils/build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-27 13:08:52,088 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 31 kolommen
2025-05-27 13:08:52,089 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend'

Day 30: 🧪 Best params: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.8}
Day 30: ✅ 167 test rows, Run: 04-13

📊 OVERALL RMSE - XGBoost Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                167  0.08
         2 2025-03-16                167  0.10
         3 2025-03-17                167  0.10
         4 2025-03-18                167  0.10
         5 2025-03-19                167  0.10
         6 2025-03-20                167  0.09
         7 2025-03-21                167  0.10
         8 2025-03-22                167  0.09
         9 2025-03-23                167  0.06
        10 2025-03-24                167  0.08
        11 2025-03-25                167  0.10
        12 2025-03-26                167  0.10
        13 2025-03-27                167  0.11
        14 2025-03-28                167  0.11
        15 2025-03-29           

In [ ]:
# Get feature importances from the trained XGBoost model
importances = model.feature_importances_
feature_names = model.feature_names_in_

# Create a DataFrame for better visualization
feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feat_imp_df)

# Optional: Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette='viridis')
plt.title('Feature Importance (XGBoost)')
plt.tight_layout()
plt.show()

In [ ]:
# Get feature importances from the last trained model
importances = model.feature_importances_
feature_names = model.feature_names_in_

# Create a DataFrame for better visualization
feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feat_imp_df)

# Optional: Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette='viridis')
plt.title('Feature Importance (XGBoost)')
plt.tight_layout()
plt.show()